# imports

In [2]:
import vectorizer as vr
import cluster as clst
import analyze as anlz
import warnings
import argparse
from LM_vectorizer import batchify, get_docs_repr
import create_vec

# import data
from dataloader import *
%matplotlib inline
def create_dir(dir):
    print(dir)
    print(os.getcwd())
    if not os.path.exists(dir):
        os.makedirs(dir)
    return os.path.abspath(dir)
# from LM_hagai import repackage_hidden

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)



import our libaries

In [3]:
corpus = Dataloader(None)

# Setup

In [4]:
labels_names = [corpus.target_names[x] for x in corpus.labels]
super_class_labels_names = corpus.super_class_labels_by_name
super_class_labels = corpus.super_class_labels
os.chdir('/home/lab/vgilad/PycharmProjects/lstm_ds_project/18_12')
num_of_documents = len(corpus.only_encoded_docs)
labels = corpus.labels
pp_docs = [' '.join([corpus.decoder[str(w)] for w in doc]) for doc in corpus.only_encoded_docs]
emails = pp_docs
# emails = corpus.raw_data
# labels = corpus.raw_labels
example_idx = 0
number_of_labels = 20  # TODO magic number
number_of_labels_super = 6  # TODO magic number

max_df = 1.  # 0.05
min_df = 0.  # 1e-4
all_k = [20, 50] #, 200]
all_vect = [vect_tfidf, vect_w2v, vect_bow]
assert (len(emails) == len(labels))
num_of_documents = len(emails)


print(corpus.raw_data[corpus.encoded_docs[example_idx][0]])



I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


parmeters

In [5]:
print(pp_docs[example_idx])


wondering car saw sports car looked late 60s early 70s doors small addition front bumper separate rest body model name engine specs production car history whatever info car


In [6]:
param = Params(vect_tfidf, clust_kneams, aff_euclidean, link_ward, min_df, max_df, 20, num_of_documents,True)


select vectorizing method


In [18]:
if (param.vectorizing == vect_bow):
    # BOW
    cv, tokenized_emails = vr.tokenizer(emails, param.min_df, param.max_df)
    # {k: v for k, v in zip(cv.get_feature_names(), sample_vec.toarray()[0]) if v > 0}
    # to get vocabulary names use: cv.get_feature_names()
    voc_names = cv.get_feature_names()
    # to get BOW for each mail use: tokenized_emails.toarray()
    emails_representation = tokenized_emails.toarray()
elif (param.vectorizing == vect_tfidf):
    # TF-IDF
    tf_idf = vr.tf_idf(emails, param.min_df, param.max_df)
    emails_representation = tf_idf.toarray()
elif (param.vectorizing == vect_w2v):
    # Word2Vec
    emails_representation = vr.BOW_w2v(emails,"w2v.pickle")
elif param.vectorizing == vect_LM:
    with open(args.checkpoint, 'rb') as f:
        if args.cuda:
            model = torch.load(f).to(device)
        else:
            model = torch.load(f, map_location='cpu').to(device)
    model.eval()
    data = batchify(corpus.only_encoded_docs, 1, device)
    emails_representation = get_docs_repr(model, data)
elif param.vectorizing == vect_gilad:
    # data = batchify(corpus.only_encoded_docs, 1, device)
    import opts

    # take only test documents (fist 1000 are val last 2000 are train)
    ntokens = len(corpus.decoder)
    num_of_documents = len(corpus.only_encoded_docs[1000:-2000])
    labels = corpus.labels[1000:-2000]
    pp_docs = [' '.join([corpus.decoder[str(w)] for w in doc]) for doc in
               corpus.only_encoded_docs]
    emails = pp_docs[1000:-2000]
    super_class_labels = corpus.super_class_labels[1000:-2000]
    super_class_labels_names = corpus.super_class_labels_by_name[1000:-2000]
    labels_names = [corpus.target_names[x] for x in labels]

    opt = opts.parse_opt()
    emails_representation, labels_not_used = create_vec.create_vec(opt)  # get numpy matrix

    # emails_representation = emails_representation[1000:-2000]
else:
    raise ValueError('vectorizing is not supported with: ' + param.clustering)

# vectorizing

Bag Of Words

In [20]:
df = anlz.genrate_tsne(emails_representation, seed=4, perplexity=30)


compute tsne with perplexity 30 and seed 4


ValueError: several sets except to list of pathes

In [ ]:
anlz.plot_tsne_df(df, (labels_names, super_class_labels_names))

# clustering

In [21]:
        if (param.clustering == clust_kneams):
            if (param.affine == 'cosine'):
                # k-means cosine dist
                clusters, kmns_class = clst.kmeans_cosine_dist(emails_representation, param.k)
            elif (param.affine == 'euclidean'):  # k-means euclidean
                clusters, kmns_class = clst.kmeans(emails_representation, param.k)
                print('kmeans sum of distanses = %f' % kmns_class.inertia_)
            else:
                raise ValueError('kmenas is not supported with affinity: ' + param.affine)
        elif (param.clustering == clust_hirarchical):
            # hirarchical
            (clusters, _) = clst.hirarchical(emails_representation, param.k, aff=param.affine, link=param.linkage)
        else:
            raise ValueError('clustering is not supported with: ' + param.clustering)

kmeans sum of distanses = 9740.223330


# analayze

In [32]:

results,results_super_class = anlz.analyze_clustering(labels, clusters, number_of_labels,super_class_labels, number_of_labels_super, calc_linkage=False)
print('accuracy = %f' % results.get_list()[0])
print('accuracy by super_class= %f' % results_super_class.get_list()[0])

TypeError: analyze_clustering() got an unexpected keyword argument 'calc_linkage'